# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [5]:
# import data
# how to import 3 files at the same time?
import pandas as pd
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv'
insurance_1 = pd.read_csv(url)
insurance_1.shape

(4008, 11)

In [6]:
import pandas as pd
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv'
insurance_2 = pd.read_csv(url)
insurance_2.shape

(996, 11)

In [7]:
import pandas as pd
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv'
insurance_3 = pd.read_csv(url)
insurance_3.shape

(7070, 11)

In [8]:
insurance_1.columns = insurance_1.columns.str.replace(" ","_").str.lower() 
insurance_1 = insurance_1.rename(columns = {"st" :"state"})
insurance_1.head(1)

,customer,state,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934


In [9]:
insurance_2.columns = insurance_2.columns.str.replace(" ", "_").str.lower()
insurance_2 = insurance_2.rename(columns = {"st" : "state"})
insurance_2.head(1)

,customer,state,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,total_claim_amount,policy_type,vehicle_class
0,GS98873,Arizona,F,Bachelor,323912.47%,16061,88,1/0/00,633.6,Personal Auto,Four-Door Car


In [10]:
insurance_3.columns = insurance_3.columns.str.replace(" ","_").str.lower()
insurance_3.head(1)

,customer,state,customer_lifetime_value,education,gender,income,monthly_premium_auto,number_of_open_complaints,policy_type,total_claim_amount,vehicle_class
0,SA25987,Washington,3479.137523,High School or Below,M,0,104,0,Personal Auto,499.2,Two-Door Car


In [11]:
# 这三个表的名字全部相同，只是在表中的顺序不同。这种情况可以
# 1）✅ Solution: Explicitly Reorder Columns Before Concatenation
# 2 🚀 Alternative: Let Pandas Handle It Automatically，This works only if all DataFrames have the exact same column names.
insurance = pd.concat([insurance_1, insurance_2, insurance_3], ignore_index=True)
insurance.shape

(12074, 11)

In [12]:
# concat 好了之后再查一下还有没有重复的列名了。
insurance.columns.duplicated()

array([False, False, False, False, False, False, False, False, False,
       False, False])

In [13]:
# 检查有无列的内容是否完全重复的。
# 上一步的检查已经包含了这一步。见某天的chatgpt。
# insurance.T.duplicated()

In [14]:
# 查看有没有重复的行。
insurance.duplicated().sum()

2939

In [15]:
# 删除重复的行,再检查shape。
insurance.drop_duplicates(inplace = True)
insurance.shape

(9135, 11)

In [16]:
insurance.dtypes
# 📌 If a Column is object Type, Does It Mean It's a String?
# 🔹 Not necessarily! In Pandas, a column with dtype=object can contain strings, but it can also contain other types like numbers, lists, dictionaries, or even mixed data types.
# 🔹Check the actual types inside the column: 
#  print(df["column"].apply(type))

customer                      object
state                         object
gender                        object
education                     object
customer_lifetime_value       object
income                       float64
monthly_premium_auto         float64
number_of_open_complaints     object
policy_type                   object
vehicle_class                 object
total_claim_amount           float64
dtype: object

In [17]:
# since Object type can contain mixed data types, i want to check in detail what each coulmn has. how to simplify the process below?
print(insurance["customer"].apply(type).unique())
print(insurance["state"].apply(type).unique())
print(insurance["gender"].apply(type).unique())
print(insurance["education"].apply(type).unique())
# 可以看出有str和float（probably NaN)

[<class 'str'> <class 'float'>]
[<class 'str'> <class 'float'>]
[<class 'float'> <class 'str'>]
[<class 'str'> <class 'float'>]


In [18]:
# 查看每一列下有多少 NaN。
insurance.isna().sum()

customer                       1
state                          1
gender                       123
education                      1
customer_lifetime_value        8
income                         1
monthly_premium_auto           1
number_of_open_complaints      1
policy_type                    1
vehicle_class                  1
total_claim_amount             1
dtype: int64

In [19]:
# replace NaNs in each column.还要再研究一下。
# for column in list(insurance.columns):
#     insurance[column].fillna("no entry",inplace = True)
#     print(insurance[column].isna().sum())
#     print(insurance[column].apply(type).unique())

In [20]:
# # replace NaNs in all the columns where dtype is object, so these can be filled with a string.
# # customer column:
# insurance["customer"] = insurance["customer"].fillna("no entry")
# print(insurance["customer"].isna().sum())

# # state column:
# insurance["state"] = insurance["state"].fillna("no entry")
# print(insurance["state"].isna().sum())

# # gender column:
# insurance["gender"] = insurance["gender"].fillna("no entry")
# print(insurance["gender"].isna().sum())

# # education
# insurance["education"] = insurance["education"].fillna("no entry")
# print(insurance["education"].isna().sum())

In [21]:
# customer column: 不是categorical。只需要replace NaN
insurance["customer"] = insurance["customer"].fillna("no entry")

In [22]:
# state column: 是categorical，需要查看是否有不恰当的值。如typo。
print(insurance["state"].unique())
# 发现需要更改
state_mapping = {"AZ":"Arizona","Cali":"California","WA":"Washington"} #create a new dictionary to save state mapping.
insurance["state"] = insurance["state"].replace(state_mapping)
print(insurance.state.unique())
insurance["state"] = insurance["state"].fillna("no entry")
print(insurance["state"].isna().sum())

['Washington' 'Arizona' 'Nevada' 'California' 'Oregon' 'Cali' 'AZ' 'WA'
 nan]
['Washington' 'Arizona' 'Nevada' 'California' 'Oregon' nan]
0


In [23]:
# customer_lifetime_value column
# dtype 一会就报错到底几个意思？
insurance["customer_lifetime_value"] = insurance["customer_lifetime_value"].str.replace("%","").astype(float).round(2)
insurance["customer_lifetime_value"]

insurance["customer_lifetime_value"] = insurance["customer_lifetime_value"].fillna(insurance["customer_lifetime_value"].mean())
insurance["customer_lifetime_value"].isna().sum()


0

In [24]:
insurance["number_of_open_complaints"] = insurance["number_of_open_complaints"].astype(str).str.split("/").str[1]
insurance["number_of_open_complaints"].unique()

array(['0', '2', '1', '3', '5', '4', nan], dtype=object)

In [25]:
# chatgpt:
'''insurance["customer_lifetime_value"] = (
    insurance["customer_lifetime_value"]
    .str.replace('%', '', regex=False)   # 去掉百分号
    .astype(float)                       # 转换为浮点数
    .round(2)                            # 保留两位小数
)
'''

'insurance["customer_lifetime_value"] = (\n    insurance["customer_lifetime_value"]\n    .str.replace(\'%\', \'\', regex=False)   # 去掉百分号\n    .astype(float)                       # 转换为浮点数\n    .round(2)                            # 保留两位小数\n)\n'

In [26]:
# income column: fillna with mode
insurance["income"].fillna(insurance["income"].mode())
insurance["income"].isna().sum()

1

In [27]:
# vehicle_class column

vehicle_class_map = {"Luxury SUV":"Luxury", "Sports Car":"Luxury","SUV" :"Luxury","Luxury Car": "Luxury"}
insurance["vehicle_class"] = insurance["vehicle_class"].replace(vehicle_class_map)
insurance.vehicle_class.unique()
insurance["vehicle_class"].fillna("no entry")
insurance["vehicle_class"].isna().sum()

1

# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [28]:
import pandas as pd
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv'
marketing = pd.read_csv(url)
marketing.shape
marketing.head()

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,A,2
1,1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,...,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,A,1
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,...,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,2
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,1
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,...,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,A,1


1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

In [29]:
marketing["sales_channel"].unique()

array(['Agent', 'Call Center', 'Branch', 'Web'], dtype=object)

In [30]:
pivot_sales_channel = marketing.pivot_table(index='sales_channel',values="customer_lifetime_value", aggfunc='sum').round(2)
pivot_sales_channel.head()

,customer_lifetime_value
sales_channel,
Agent,33057887.85
Branch,24359201.21
Call Center,17364288.37
Web,12697632.90


2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

In [31]:
marketing["education"].unique()
marketing["gender"].unique()

array(['M', 'F'], dtype=object)

In [32]:
pivot_gender_edu = marketing.pivot_table(index = "gender", columns = "education", values = "customer_lifetime_value",aggfunc = "mean").round(2)
pivot_gender_edu

education,Bachelor,College,Doctor,High School or Below,Master
gender,,,,,
F,7874.27,7748.82,7328.51,8675.22,8157.05
M,7703.60,8052.46,7415.33,8149.69,8168.83


## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [33]:
complaint_summary = marketing.pivot_table(
    index='month',
    columns='policy_type',
    values='number_of_open_complaints',
    aggfunc='count',
    fill_value=0
)
long_format = complaint_summary.stack().reset_index()

In [34]:
insurance.head()

,customer,state,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount
0,RB50392,Washington,NaN,Master,7.802640e+05,0.0,1000.0,0,Personal Auto,Four-Door Car,2.704934
1,QZ44356,Arizona,F,Bachelor,6.979536e+05,0.0,94.0,0,Personal Auto,Four-Door Car,1131.464935
2,AI49188,Nevada,F,Bachelor,1.288743e+06,48767.0,108.0,0,Personal Auto,Two-Door Car,566.472247
3,WW63253,California,M,Bachelor,7.645862e+05,0.0,106.0,0,Corporate Auto,Luxury,529.881344
4,GA49547,Washington,M,High School or Below,5.363077e+05,36357.0,68.0,0,Personal Auto,Four-Door Car,17.269323
